In [ ]:
import os
from pathlib import Path

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Building Sensitivity Analysis


The aim of this tutorial is to provide a workflow for building thermal simulation sensitivity analysis using
[__EnergyPlus__](https://energyplus.net), __energytool__, and __corrAI__ sensitivity module :https://github.com/BuildingEnergySimulationTools/corrai/blob/main/corrai/sensitivity.py



## Introduction

Sensitivity Analysis methods are mathematical method that quantify the impact of and uncertain parameters on a specific metrics.
Various methods exists such as Morris or Sobol.

During building conception workflow, sensitivity analysis can have various benefits:
- Screen out a number of irrelevant conception variables to focus on the important ones (does the solar absorption of the partition glass-wool have a relevant impact on building heat needs ?)
- Sort  relevant uncertain parameter by influence on the observed metrics.
- Help you quantify the relative importance of the modeled physical phenomenons

In this example, the use case is an old building retrofitting. The objective is to insulate the south facade using double skin.

|               Figure 1: Building picture               |            Figure 2: Building thermal model            |
|:------------------------------------------------------:|:------------------------------------------------------:|
| <img src="resources/building_photo.png"  height="300"> | <img src="resources/building_model.png"  height="300"> |

The designer wants to know the impact of the following variables on the building **heat needs** and on the **thermal comfort**:
- Double skin glazing thermal properties : Solar Heat Gain Coefficient (SHGC), thermal conductivity coefficient ($U_{value}$)
- Envelope glazing thermal properties : Solar Heat Gain Coefficient (SHGC), thermal conductivity coefficient ($U_{value}$)
- Envelope insulation thickness
- Air infiltration coefficient Q4Pa [m<sup>3</sup>/h.m² @4Pa]

To answer these questions, we will use an EnergyPlus building model, energytool and corrAI classes to perform Morris & Sobol sensitivity analyses.

## Building modeling

In energytool, the <code>Building</code> class is used to simulate HVAC systems through pre-process and post process methods.
The <code>Building</code> holds and idf file. The user specify hvac system using the <code>system</code> module.

In [ ]:
from energytool.building import Building, SimuOpt

The path of the idd file of EnergyPlus must be given to the <code>Building</code> class.
Be careful idf file E+ version and idd file version must match.

In [ ]:
Building.set_idd(Path(r"C:\EnergyPlusV9-4-0"))

Now we instantiate a building with an idf file representing the building thermal model.
The idf can be generated manually or using a software (Openstudio, DesignBuilder).
Keep in mind that the main advantage of the energytool <code>Building</code> class, is to simplify hvac system modeling using pre-process and post-process methods.
Thus, we recommend using <code>IdealLoadAirSystem</code> to model HVAC and Domestic Hot Water production (DHW).

In [ ]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/tuto_as.idf")

The <code>infos()</code> method display information on the building object

In [ ]:
building

It is time to specify the building hvac equipments.
Let's use the one present in the <code>system</code> module.
Note that you can use custom class as long as they contain a <code>pre_process()</code> and a <code>post_process()</code> methods

In this example, we will only use a boiler with a cop of 1 as we want to work with building heating needs.
For more information on building system, see the dedicated tutorial.

In [ ]:
from energytool.system import Sensor, HeaterSimple

In [ ]:
building.add_system(HeaterSimple(name="IdealBoiler", zones = ["RX2:Zone1", "RX1:Zone1", "RDC:Zone1"], cop=1))
building.add_system(Sensor(name="Temperatures", variables="Zone Mean Air Temperature", key_values="*"))

We specify the parameters we are uncertain about using the class <code>UncertainParameter</code> from the <code>energytool.parameter</code> module.

In [ ]:
from corrai.base.parameter import Parameter

A <code>list</code> of dictionaries is used to define uncertain parameters in the model. 
Each dictionary represents one uncertain parameter and follows the structure defined by the <code>Parameter</code> enum.
The example below defines the Solar Heat Gain Coefficient (SHGC) of the external glazing:
```
{
Parameter.NAME: "idf.WindowMaterial:SimpleGlazingSystem.Simple DSF_ext_south_glazing - 1002.Solar_Heat_Gain_Coefficient",
Parameter.INTERVAL: [0.3, 0.7],
Parameter.TYPE: "Real",
} 
```
<code>Parameter.NAME</code>: Full path to the parameter in the IDF model, following the format:
<code>"idf.<idf_object>.<name>.<field>"</code>.
Use <code>"*"</code> for <code>name</code> if the parameter applies to all objects of the given type.

<code>Parameter.INTERVAL</code>: The lower and upper bounds of the uncertainty interval.
For discrete uncertainties, this should be a list of all possible values.

<code>Parameter.TYPE</code>: Indicates the type of variable:
<code>"Real"</code>, <code>"Integer"</code>, <code>"Binary"</code>, or <code>"Choice"</code>.

- Use <code>"Real"</code> for continuous parameters like SHGC or U-Factor.
- Use <code>"Integer"</code> for only integer values between the lower and upper bounds
- Use <code>"Choice"</code> for categorical parameters (works in specific cases if more than two values are given. Not Morris)
- use <code>"Binary"</code> for 0 - 1 values

You can define as many uncertain parameters as needed using this structure.

In [ ]:
uncertain_param_list = [
    {
        Parameter.NAME: "idf.WindowMaterial:SimpleGlazingSystem.Simple DSF_ext_south_glazing - 1002.Solar_Heat_Gain_Coefficient",
        Parameter.INTERVAL: [0.5, 0.8],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "idf.WindowMaterial:SimpleGlazingSystem.Simple DSF_ext_south_glazing - 1002.UFactor",
        Parameter.INTERVAL: [0.5, 0.8],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "idf.Material.Wall_insulation_.1.Thickness",
        # Parameter.INTERVAL: [0.1, 0.2, 0.4, 0.6],
        Parameter.INTERVAL: [0.1, 0.6],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "idf.AirflowNetwork:MultiZone:Surface:Crack.*.Air_Mass_Flow_Coefficient_at_Reference_Conditions",
        Parameter.INTERVAL: [0.05, 0.5],
        Parameter.TYPE: "Real",
    },
]

Import the sensitivity analysis class <code>SAnalysis</code>
As a minimal configuration, it requires the <code>Building</code> instance, en sensitivity analysis method and the previously defined list of uncertain parameter.

In [ ]:
from corrai.sensitivity import SAnalysis, Method

## Morris method
To screen out parameters or to have a first estimation of the uncertain parameters rank without running too many simulation, it is often a good idea to use the Morris method.

In [ ]:
sa_analysis = SAnalysis(
    parameters_list=uncertain_param_list,
    method=Method.MORRIS,
)

The <code>draw_sample</code> method of <code>SAnalysis</code> draws parameters values according to the <code>parameters</code> list. The sampling method depends on the <code>sensitivity_method</code>. For Morris a One At a Time method is used (OAT). See [SALib documentation](https://salib.readthedocs.io/en/latest/index.html) for more information

The number of trajectories is set to 15.

In [ ]:
sa_analysis.draw_sample(n=5)

Sampling results are stored in <code>sample</code>. Columns corresponds to parameters.
Index lines corresponds to a configuration (combination of parameters values)

In [ ]:
sa_analysis.sample.head()

In [ ]:
len(sa_analysis.sample)

<code>run_simulations</code> method runs the 105 simulations

In [ ]:
SIM_OPTIONS = {
    SimuOpt.EPW_FILE.value: Path(TUTORIAL_DIR) / r"resources/FRA_Bordeaux.075100_IWEC.epw",
    SimuOpt.OUTPUTS.value: "SYSTEM|SENSOR",
    SimuOpt.START.value: "2025-01-01",
    SimuOpt.STOP.value: "2025-01-19",
    SimuOpt.VERBOSE.value: "v", 
}

In [ ]:
building

In [ ]:
sa_analysis.evaluate(
    model = building, 
    simulation_options=SIM_OPTIONS,
)

We can plot all simulations in one graph and compare the simulated internal temperature or heating energy. Argument show_legends can be set to True if you want see associated parameters values.

In [ ]:
from corrai.sensitivity import plot_sample

plot_sample(
    sample_results=sa_analysis.sample_results,
    indicator="RX1:ZONE1_Zone Mean Air Temperature",
    show_legends=True,
    y_label="Temperature [°C]",
    x_label="Date",
)

In [ ]:
from corrai.sensitivity import plot_sample

plot_sample(
    sample_results=sa_analysis.sample_results,
    # indicator="RX1:ZONE1_Zone Mean Air Temperature",
    indicator="HEATING_Energy_[J]",
    show_legends=False,
    y_label="HEATING_Energy_[J]",
    x_label="Date",
)

In [ ]:
from corrai.metrics import cv_rmse, nmbe
import numpy as np

sa_analysis.analyze(
    # indicator="RX1:ZONE1_Zone Mean Air Temperature",
    indicator="HEATING_Energy_[J]",
    # agg_method=np.mean,
    agg_method=np.sum,
)

In [ ]:
sa_analysis.sensitivity_results

In [ ]:
sa_analysis.calculate_sensitivity_indicators()

Sensitivity index results are stored in <code>sensitivity_results</code>.
Pre-formatted figure for Morris results is available using <code>plot_morris_scatter</code>

In [ ]:
from corrai.sensitivity import plot_morris_scatter 
plot_morris_scatter(salib_res=sa_analysis.sensitivity_results, title='Elementary effects', unit='J', autosize=True) 

In the figure above:
- Circle size indicate the total effect of the parameter on the chosen indicator. The bigger, the more influential.
- The x axis is the mean elementary effect of the parameters. It represents "linear" effect of the parameter.
- The y axis is the standard deviation. It represents interactions between parameters and non linearities.
- The 3 lines separates the figure in 4 regions. From the closer to the x axis : linear, monotonic, almost monotonic and non-linear and/or non-monotonic effects. See [publication](http://www.ibpsa.org/proceedings/BSO2016/p1101.pdf) for more details
- The segment represent the uncertainty on the sensitivity index calculation.

In this use case. Several conclusions can be drawn:
- 4 parameters have an influence on the chosen indicator. Two indicators can be neglected.
- The 4 main parameters have an almost linear influence on the indicator
- The confidence on the sensitivity index calculation is high

## Sobol method

Sobol index indicates the contribution of each uncertain parameters to the variance of the output indicator.
It is a more accurate method to quantify the effect of an uncertain parameter. The second order index also gives more information on the effect of parameters interactions.
... But it comes at a much higher computational cost.
In energytool, the index are computed using SALib. The method gives an estimation of the index value. It reduces the simulation sample size.

Below is an example of a <code>SAnalisys</code> configuration to perform Sobol index calculation
It is very similar to Morris

In [ ]:
sa_analysis_sob = SAnalysis(
    parameters_list=uncertain_param_list,
    method=Method.SOBOL,
)

In [ ]:
# Salib command an n = 2^x. In this case x shall be >= 6
sa_analysis_sob.draw_sample(n=2**2)
len(sa_analysis_sob.sample)

In [ ]:
sa_analysis_sob.evaluate(
    model = building, 
    simulation_options=SIM_OPTIONS,
)

Similarly to Morris, a function is designed to plot preformatted Sobol total index graph

In [ ]:
from corrai.sensitivity import plot_sobol_st_bar

In [ ]:
from corrai.metrics import cv_rmse, nmbe
import numpy as np

sa_analysis_sob.analyze(
    indicator="RX1:ZONE1_Zone Mean Air Temperature",
    agg_method=np.mean,
)

sa_analysis_sob.calculate_sensitivity_indicators()
plot_sobol_st_bar(sa_analysis_sob.sensitivity_results)

 In this use case, the Sobol method sorted the uncertain parameters in the same order as Morris.
The Sobol total index represent an uncertain parameter single effect plus the sum of all its interactions on the considered indicator.
The uncertainty bar shows the confidence interval of the index value.

The sum of all the index shall be equal to one.
Salib computes an estimation of this index. 

In [ ]:
sa_analysis_sob.sensitivity_results["ST"].sum()

The confidence intervals of the index overlap and the sum is much higher than 1. A bigger sample is necessary to draw conclusion, and should greatly improve the results.